In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
import os

import shutil

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from tqdm import tqdm

In [ ]:
#!unzip -q /content/lnu-deep-learn-1-image-classification-2025.zip -d /content/fast_data

#!rm /content/lnu-deep-learn-1-image-classification-2025.zip

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовуємо пристрій: {device}")
if device.type == 'cuda':
    print(f"Назва відеокарти: {torch.cuda.get_device_name(0)}")

data_dir = 'fast_data'
train_dir = os.path.join(data_dir, 'train')

Використовуємо пристрій: cuda
Назва відеокарти: Tesla T4


In [ ]:
os.chdir('/content')

print("Поточна папка:", os.getcwd())
print("Вміст папки:", os.listdir())

Поточна папка: /content
Вміст папки: ['.config', 'drive', 'fast_data', 'sample_data']


In [ ]:
classes_path = os.path.join(data_dir, 'classes.txt')

with open(classes_path, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

num_classes = len(class_names)

print(f"Кількість класів: {num_classes}")
print(f"Перші 5 класів для прикладу: {class_names[:5]}")

Кількість класів: 100
Перші 5 класів для прикладу: ['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO']


In [ ]:
train_dir = '/content/fast_data/train'

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,       
    shuffle=True,
    num_workers=2,       
    pin_memory=True      
)

print(f"Дані завантажено з: {train_dir}")
print(f"Класів: {len(train_dataset.classes)}")

Дані завантажено з: /content/fast_data/train
Класів: 100


In [ ]:
print("Завантажуємо AlexNet...")
model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)

input_features = 9216    
hidden_size = 512        

for param in model.parameters():
    param.requires_grad = False

layers_to_unfreeze = [8, 10]
for i in layers_to_unfreeze:
    for param in model.features[i].parameters():
        param.requires_grad = True


model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(input_features, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),


    nn.Linear(4096, hidden_size), 
    nn.ReLU(inplace=True),

    nn.Linear(hidden_size, num_classes) 
)


model = model.to(device)

print(f"✅ Класифікатор оновлено: Додано прихований шар на {hidden_size} нейронів.")
print("Новий класифікатор:", model.classifier)

for param in model.classifier.parameters():
    param.requires_grad = True

Завантажуємо AlexNet...
✅ Класифікатор оновлено: Додано прихований шар на 512 нейронів.
Новий класифікатор: Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=512, out_features=100, bias=True)
)


In [ ]:
criterion = nn.CrossEntropyLoss()
best_acc = 0.0
patience = 5                 
epochs_without_improvement = 0
checkpoint_path = '/content/drive/MyDrive/butterfly_project/alexnet_unfreeze_butterfly_model.pth'


EPOCHS_PHASE_1 = 10         
EPOCHS_PHASE_2 = 20        
TOTAL_EPOCHS = EPOCHS_PHASE_1 + EPOCHS_PHASE_2



optimizer_1 = optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler_1 = lr_scheduler.StepLR(optimizer_1, step_size=3, gamma=0.5)


print("\n--- ФАЗА 1/2: НАВЧАННЯ ТІЛЬКИ НОВОГО КЛАСИФІКАТОРА (10 ЕПОХ) ---")

current_optimizer = optimizer_1
current_scheduler = scheduler_1

for epoch in range(EPOCHS_PHASE_1):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in tqdm(train_loader, desc=f"Фаза 1 Епоха {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)

        current_optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        current_optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()

    current_scheduler.step()
    epoch_acc = 100 * correct_preds / total_preds
    epoch_loss = running_loss / len(train_loader)
    current_lr = current_optimizer.param_groups[0]['lr']

    print(f"\n🏁 ФАЗА 1, Епоха [{epoch+1}/{EPOCHS_PHASE_1}] | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | LR: {current_lr:.6f}")


    if epoch_acc > best_acc:
        best_acc = epoch_acc
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print(f"✅ Рекорд! Модель збережено (Acc: {best_acc:.2f}%)")
    else:
        epochs_without_improvement += 1
        print(f"⏹️ Точність не покращилась. Очікування: {epochs_without_improvement}/{patience}")

    if epochs_without_improvement >= patience:
        print(f"🛑 Рання зупинка! Немає покращення {patience} епох.")
        break



if epochs_without_improvement < patience:

    print("\n--- ПЕРЕХІД ДО ФАЗИ 2/2: ТОНКЕ НАЛАШТУВАННЯ ВСІЄЇ МОДЕЛІ ---")
    layers_to_unfreeze = [0, 3, 6] 
    for i in layers_to_unfreeze:
        for param in model.features[i].parameters():
            param.requires_grad = True

    current_optimizer = optim.Adam(model.parameters(), lr=0.0001) 
    current_scheduler = lr_scheduler.StepLR(current_optimizer, step_size=3, gamma=0.5)

    epochs_without_improvement = 0

    for epoch in range(EPOCHS_PHASE_2):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in tqdm(train_loader, desc=f"Фаза 2 Епоха {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)

            current_optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            current_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_preds += labels.size(0)
            correct_preds += (predicted == labels).sum().item()

        current_scheduler.step()
        epoch_acc = 100 * correct_preds / total_preds
        epoch_loss = running_loss / len(train_loader)
        current_lr = current_optimizer.param_groups[0]['lr']

        print(f"\n🔥 ФАЗА 2, Епоха [{epoch+1}/{EPOCHS_PHASE_2}] | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | LR: {current_lr:.6f}")


        if epoch_acc > best_acc:
            best_acc = epoch_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), checkpoint_path)
            print(f"✅ Рекорд! Модель збережено (Acc: {best_acc:.2f}%)")
        else:
            epochs_without_improvement += 1
            print(f"⏹️ Точність не покращилась. Очікування: {epochs_without_improvement}/{patience}")

        if epochs_without_improvement >= patience:
            print(f"🛑 Рання зупинка! Немає покращення {patience} епох.")
            break


print("\n🏆 ФІНАЛ. Тренування завершено!")
print(f"Найкраща точність (збережена): {best_acc:.2f}%")


--- ФАЗА 1/2: НАВЧАННЯ ТІЛЬКИ НОВОГО КЛАСИФІКАТОРА (10 ЕПОХ) ---


Фаза 1 Епоха 1: 100%|██████████| 171/171 [00:20<00:00,  8.37it/s]



🏁 ФАЗА 1, Епоха [1/10] | Loss: 2.3287 | Acc: 45.91% | LR: 0.001000
✅ Рекорд! Модель збережено (Acc: 45.91%)


Фаза 1 Епоха 2: 100%|██████████| 171/171 [00:22<00:00,  7.69it/s]



🏁 ФАЗА 1, Епоха [2/10] | Loss: 0.7769 | Acc: 77.80% | LR: 0.001000
✅ Рекорд! Модель збережено (Acc: 77.80%)


...


🔥 ФАЗА 2, Епоха [18/20] | Loss: 0.0105 | Acc: 99.70% | LR: 0.000002
⏹️ Точність не покращилась. Очікування: 1/5


Фаза 2 Епоха 19: 100%|██████████| 171/171 [00:22<00:00,  7.47it/s]



🔥 ФАЗА 2, Епоха [19/20] | Loss: 0.0103 | Acc: 99.73% | LR: 0.000002
⏹️ Точність не покращилась. Очікування: 2/5


Фаза 2 Епоха 20: 100%|██████████| 171/171 [00:22<00:00,  7.46it/s]


🔥 ФАЗА 2, Епоха [20/20] | Loss: 0.0103 | Acc: 99.71% | LR: 0.000002
⏹️ Точність не покращилась. Очікування: 3/5

🏆 ФІНАЛ. Тренування завершено!
Найкраща точність (збережена): 99.76%


In [ ]:
class ButterflyTestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_name


test_dir = '/content/fast_data/test'


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_dataset = ButterflyTestDataset(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) # shuffle=False важливо!

print(f"Знайдено тестових зображень: {len(test_dataset)}")

Знайдено тестових зображень: 2728


In [ ]:
try:
    class_names = train_dataset.classes
except NameError:
    print("Відновлюємо список класів...")
    temp_data = datasets.ImageFolder('/content/fast_data/train')
    class_names = temp_data.classes

print(f"Всього класів: {len(class_names)}")


all_ids = []
all_preds = []

model.eval() 
model.to(device)

print("⏳ Починаю передбачення...")

with torch.no_grad(): 
    for inputs, filenames in tqdm(test_loader):
        inputs = inputs.to(device)


        outputs = model(inputs)


        _, predicted_indices = torch.max(outputs, 1)

        for i in range(len(filenames)):
            file_id = os.path.splitext(filenames[i])[0]
            all_ids.append(file_id)

            class_idx = predicted_indices[i].item()
            predicted_label = class_names[class_idx]
            all_preds.append(predicted_label)


submission_df = pd.DataFrame({
    'id': all_ids,
    'label': all_preds
})


csv_path = '/content/drive/MyDrive/butterfly_project/submission_alexnet_unfreeze.csv'
submission_df.to_csv(csv_path, index=False)

print(f"\n✅ Готово! Файл збережено: {csv_path}")
print("Ось перші 5 рядків для перевірки:")
print(submission_df.head())

Всього класів: 100
⏳ Починаю передбачення...


100%|██████████| 86/86 [00:05<00:00, 14.80it/s]


✅ Готово! Файл збережено: /content/drive/MyDrive/butterfly_project/submission_alexnet_unfreeze.csv
Ось перші 5 рядків для перевірки:
                                 id              label
0  7bf8001f46fd4b709a367ddfa91d4971  COMMON WOOD-NYMPH
1  d0058361d71b4cb4a498d4154d3cf5ff             ADONIS
2  a8be47bb65c2463d9e9f9f291e6dce09            MONARCH
3  738c1ed1771345af87465ce9868edb4b   GREEN HAIRSTREAK
4  7d8c94c8ad014295be46844983b76b87        RED CRACKER
